Install the OpenAI libray

In [173]:
%pip install openai  python-dotenv  typing-extensions



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Load neccesary environment variables

In [174]:
# Import the OpenAI API key from the .env file
import os # Import the OS module
from dotenv import load_dotenv # Import the load_dotenv function from the dotenv module
load_dotenv() # Load the .env file

True

Get variables from the environment

In [175]:
# Get the OpenAI API key from the environment
openai_api_key = os.getenv('OPEN_AI_API_KEY') # Get the OpenAI API key from the environment



Create connection to OpenAI

In [176]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key) # Create a new OpenAI client

Initiate GPT assistant

In [177]:
# Create a new assistant

assistant = client.beta.assistants.create(
    name="Kudu AI",
    instructions="You are a medical assistant and helps patients with questions around their medication. You can provide information about the medication, side effects, and how to take it. You can also help patients find the right medication for their condition.", # Provide instructions to the AI
    tools=[{"type": "file_search"}], # Define the tools that the AI can use
    model="gpt-4o", # Use the GPT-4 model and enable chat mode
    )

Uploads drug database to Vector Store

In [178]:
# Create a vector store called "Drug Database"
vector_store = client.beta.vector_stores.create(name="Drug Database")
 
# Ready the files for upload to OpenAI
file_paths = ["files/DrugsbyNDCClass3_090809.pdf", ]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


Update assistant to use new Vector Store

In [179]:
# Add the vector store to the assistant
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Initialize thread

In [186]:
# Create a new conversation thread
thread = client.beta.threads.create() 

Create message between assistant and user

In [187]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Create Ongoing Chat with Stream
while True:
    message_input = input("You: ")

    # Send the user's message to the AI
    user_message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=message_input
    )

    # Stream the assistant's response
    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()


assistant > file_search


assistant > Tylenol, which is commonly known as acetaminophen, is an over-the-counter medication used as a pain reliever and fever reducer. It falls under several product variations, including Extra Strength Tylenol, Tylenol PM, and Tylenol Arthritis, among others【4:0†source】【4:2†source】【4:10†source】【4:13†source】【4:15†source】. 

Acetaminophen is categorized under the major therapeutic class code 1700 and is utilized for its analgesic (pain-relieving) and antipyretic (fever-reducing) properties. It is generally recommended for conditions such as headaches, muscle aches, arthritis, backaches, toothaches, colds, and fevers【4:0†source】【4:17†source】.

For specific therapeutic uses:
- **Tylenol PM**: Combines acetaminophen with a sleep aid for nighttime pain relief【4:8†source】.
- **Tylenol Arthritis**: Formulated for long-lasting relief of arthritis pain with an extended-release feature【4:10†source】.
- **Children's Tylenol**: Specially formulated for pediatric use 